In [13]:
import requests
import json
import shutil
from tqdm import tqdm
from glob import glob
import numpy as np

In [2]:
tracks = json.loads(open('../data/tracks.json').read())

In [ ]:
tracks

In [ ]:
for id, track in tqdm(tracks.items()):
    if 'preview_url' in track:
        url = track['preview_url']
        filename = 'data/previews/{}.mp3'.format(id)
        with requests.get(url, stream=True) as r:
            with open(filename, 'wb') as f:
                shutil.copyfileobj(r.raw, f)
        print('Downloaded {}'.format(filename))
    else:
        print('No preview for {}'.format(id))

In [4]:
%%bash 
ls -1 ../data/previews/*.mp3 2>/dev/null | wc -l
# I, truly, am the greatest developer of all time.

822


In [5]:
audio_files = glob('../data/previews/*.mp3')

In [6]:
len(audio_files)

822

In [7]:
from embed import embed_audio

2024-05-26 12:07:06.003431: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-26 12:07:06.003657: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-26 12:07:06.006065: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-26 12:07:06.036069: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-26 12:07:06.636352: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

In [ ]:
audio_embeddings = {}
for audio_file in tqdm(audio_files):
    track_id = audio_file.split('/')[-1].split('.')[0]
    embedding = embed_audio(audio_file, postprocess=True)
    audio_embeddings[track_id] = embedding

In [12]:
track_id_to_key = json.loads(open('../data/track_ids_mapping.json').read())

In [14]:
audio_np = np.zeros((len(audio_embeddings), 128))

In [16]:
for id, embedding in audio_embeddings.items():
    key = int(track_id_to_key[id])
    audio_np[key] = embedding

In [18]:
np.save('../data/audio_np.npy', audio_np)

In [19]:
for id, embedding in audio_embeddings.items():
    audio_embeddings[id] = embedding.tolist()

In [20]:
json.dump(audio_embeddings, open('../data/audio_embeddings.json', 'w'))

In [21]:
import faiss

In [22]:
index = faiss.IndexFlatL2(128)

In [23]:
index.add(audio_np)

In [24]:
faiss.write_index(index, '../data/audio_index.faiss')